# ConversionCentral Managed Profiling
Run this notebook from a Databricks Repo so backend deployments control profiling logic.

In [ ]:
# Collect parameters passed by the FastAPI backend


# Each widget is declared up front so Databricks jobs can safely supply overrides.
dbutils.widgets.text("table_group_id", "")
dbutils.widgets.text("profile_run_id", "")
dbutils.widgets.text("data_quality_schema", "")
dbutils.widgets.text("payload_path", "")
dbutils.widgets.text("payload_base_path", "")
dbutils.widgets.text("callback_url", "")
dbutils.widgets.text("callback_base_url", "")
dbutils.widgets.text("callback_token", "")
dbutils.widgets.text("payload_storage", "")
dbutils.widgets.text("callback_behavior", "")
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema_name", "")
dbutils.widgets.text("connection_id", "")
dbutils.widgets.text("connection_name", "")
dbutils.widgets.text("system_id", "")
dbutils.widgets.text("project_key", "")
dbutils.widgets.text("http_path", "")


from datetime import datetime
import json
import requests
from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()


_NOTEBOOK_STAGE_SEQUENCE = (
    ("parameters", "Cell 2 (widget and Spark initialization)"),
    ("profiling", "Cell 3 (profiling logic)"),
    ("payload_persistence", "Cell 4 (payload persistence and callbacks)"),
    ("metadata_helpers", "Cell 5 (metadata helper definitions)"),
    ("finalization", "Cell 6 (final persistence and callbacks)")
)
_NOTEBOOK_STAGE_LOOKUP = {label: (idx, hint) for idx, (label, hint) in enumerate(_NOTEBOOK_STAGE_SEQUENCE)}
_STAGE_SYMBOL_REQUIREMENTS = {
    "parameters": ("table_group_id", "profile_run_id", "dq_schema"),
    "profiling": ("MAX_COLUMNS_TO_PROFILE",),
    "payload_persistence": ("_resolve_payload_storage_mode", "_payload_storage_is_artifact"),
    "metadata_helpers": ("_persist_results_to_metadata",),
}


def _ensure_notebook_stage(stage_label: str) -> None:
    if stage_label not in _NOTEBOOK_STAGE_LOOKUP:
        raise ValueError(f"Unknown notebook stage '{stage_label}'.")
    stage_index, stage_hint = _NOTEBOOK_STAGE_LOOKUP[stage_label]
    for prior_label, prior_hint in _NOTEBOOK_STAGE_SEQUENCE[:stage_index]:
        required_symbols = _STAGE_SYMBOL_REQUIREMENTS.get(prior_label, ())
        missing = [symbol for symbol in required_symbols if symbol not in globals()]
        if missing:
            missing_list = ", ".join(sorted(missing))
            raise RuntimeError(
                "Profiling notebook Cells 1-6 must run sequentially. "
                f"Run {prior_hint} before {stage_hint} (missing: {missing_list}).",
            )


table_group_id = dbutils.widgets.get("table_group_id")
profile_run_id = dbutils.widgets.get("profile_run_id")
dq_schema = (dbutils.widgets.get("data_quality_schema") or "").strip()
raw_payload_path = (dbutils.widgets.get("payload_path") or "").strip()
payload_path = raw_payload_path or None
payload_base_path = (dbutils.widgets.get("payload_base_path") or "").strip() or None
callback_url = (dbutils.widgets.get("callback_url") or "").strip() or None
callback_base_url = (dbutils.widgets.get("callback_base_url") or "").strip() or None
callback_token = (dbutils.widgets.get("callback_token") or "").strip() or None
connection_catalog = (dbutils.widgets.get("catalog") or "").strip()
connection_schema = (dbutils.widgets.get("schema_name") or "").strip()


if not table_group_id or not profile_run_id:
    raise ValueError("Required widgets missing: table_group_id/profile_run_id")
if not dq_schema:
    raise ValueError("Data quality schema widget is required for profiling runs.")


_ensure_notebook_stage("parameters")

In [ ]:
# Profile the tables registered for this table group and build the result payload.
from datetime import datetime
import re
from contextlib import suppress
from typing import Iterable


if "_ensure_notebook_stage" not in globals():
    raise RuntimeError("Profiling notebook Cells 1-6 must run sequentially; run Cell 2 before profiling.")


_ensure_notebook_stage("profiling")


import datetime as dt
import hashlib
import json
import math


from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.utils import AnalysisException


MAX_COLUMNS_TO_PROFILE = 25
NULL_RATIO_ALERT_THRESHOLD = 0.5
HIGH_NULL_RATIO_THRESHOLD = 0.9
VALUE_DISTRIBUTION_LIMIT = 25
VALUE_DISTRIBUTION_DISTINCT_THRESHOLD = 1000
VALUE_DISTRIBUTION_MAX_ROWS = 5_000_000
MAX_VALUE_DISPLAY_LENGTH = 256


PROFILE_COLUMN_FIELDS = [
    "profile_run_id",
    "schema_name",
    "table_name",
    "column_name",
    "qualified_name",
    "data_type",
    "general_type",
    "ordinal_position",
    "row_count",
    "null_count",
    "non_null_count",
    "distinct_count",
    "min_value",
    "max_value",
    "avg_value",
    "stddev_value",
    "median_value",
    "p95_value",
    "true_count",
    "false_count",
    "min_length",
    "max_length",
    "avg_length",
    "non_ascii_ratio",
    "min_date",
    "max_date",
    "date_span_days",
    "metrics_json",
    "generated_at",
]


...

In [ ]:
# Persist payload and call back into the API


if "_ensure_notebook_stage" not in globals():
    raise RuntimeError("Profiling notebook Cells 1-6 must run sequentially; run prior cells before payload persistence.")


_ensure_notebook_stage("payload_persistence")


from datetime import datetime
import re
import socket
from contextlib import suppress
from functools import lru_cache
from urllib.parse import urlparse, urlunparse


from pyspark.sql import functions as F
from pyspark.sql.utils import AnalysisException


DEFAULT_PRIVATE_PAYLOAD_ROOT = "dbfs:/tmp/conversioncentral/profiles"
DEFAULT_DRIVER_PAYLOAD_ROOT = "file:/databricks/driver/conversioncentral/profiles"
DEFAULT_CALLBACK_BEHAVIOR = "metadata_only"


DEFAULT_PAYLOAD_STORAGE_MODE = "inline"




DBFS_DISABLED_MESSAGES = ("public dbfs root is disabled", "access is denied")
DRIVER_DISABLED_MESSAGES = ("local filesystem access is forbidden", "workspacelocalfilesystem")
URI_SCHEME_PATTERN = re.compile(r"^[a-z][a-z0-9+.\-]*:/", re.IGNORECASE)
_DBFS_REDIRECT_NOTICE_EMITTED = False
_STORAGE_DISABLED_NOTICE_EMITTED = False




def _looks_like_dns_failure(error: BaseException) -> bool:
    """Detect DNS resolution failures from nested request exceptions."""
    current = error
    while current:
        if isinstance(current, socket.gaierror):
            return True
        name = current.__class__.__name__.lower()
        if "nameresolution" in name:
            return True
        message = str(current).lower()
        if "temporary failure in name resolution" in message:
            return True
        current = getattr(current, "__cause__", None) or getattr(current, "__context__", None)
    return False




def _rewrite_heroku_app_host(url: str | None) -> str | None:
    """Fallback to canonical Heroku hostname when review-app hosts fail DNS."""
    if not url:
        return None
    parsed = urlparse(url)
    host = parsed.hostname
    if not host:
        return None
    match = re.match(r"^(?P<base>[a-z0-9-]+?)-[0-9a-f]{12}\.herokuapp\.com$", host)
    if not match:
        return None
    canonical_host = f"{match.group('base')}.herokuapp.com"
    netloc = canonical_host
    if parsed.port:
        netloc = f"{canonical_host}:{parsed.port}"
    if parsed.username:
        auth = parsed.username
        if parsed.password:
            auth = f"{auth}:{parsed.password}"
        netloc = f"{auth}@{netloc}"
    scheme = parsed.scheme or "https"
    if scheme.lower() == "http":
        scheme = "https"
    return urlunparse(parsed._replace(netloc=netloc, scheme=scheme))

In [ ]:
# Column/value persistence helpers and overrides
import datetime as dt
from datetime import datetime


if "_ensure_notebook_stage" not in globals():
    raise RuntimeError("Profiling notebook Cells 1-6 must run sequentially; run earlier cells before defining metadata helpers.")


_ensure_notebook_stage("metadata_helpers")


def _escape_identifier(identifier: str) -> str:
    cleaned = (identifier or "").strip().replace("`", "")
    if not cleaned:
        raise ValueError("Metadata identifiers cannot be empty.")
    return f"`{cleaned}`"


def _metadata_schema_reference() -> str:
    if not dq_schema:
        raise ValueError("data_quality_schema widget must be set before resolving metadata tables.")
    catalog = (connection_catalog or "").strip()
    if catalog:
        return f"{_escape_identifier(catalog)}.{_escape_identifier(dq_schema)}"
    return _escape_identifier(dq_schema)


def _metadata_table(table_name: str) -> str:
    return f"{_metadata_schema_reference()}.{_escape_identifier(table_name)}"


def _first_non_empty(*values):
    for value in values:
        if isinstance(value, str):
            candidate = value.strip()
            if candidate:
                return candidate
        elif value is not None:
            return value
    return None


def _coerce_int(value):
    if value is None:
        return None
    if isinstance(value, bool):
        return int(value)
    if isinstance(value, int):
        return value
    if isinstance(value, float):
        if not math.isfinite(value):
            return None
        return int(round(value))
    if isinstance(value, str):
        candidate = value.strip().replace(",", "")
        if not candidate:
            return None
        try:
            if "." in candidate:
                return int(float(candidate))
            return int(candidate)
        except ValueError:
            return None
    return None


def _coerce_float(value):
    if value is None:
        return None
    if isinstance(value, bool):
        return float(value)
    if isinstance(value, (int, float)):
        numeric = float(value)
        if math.isfinite(numeric):
            return numeric
        return None
    if isinstance(value, str):
        candidate = value.strip().replace(",", "")
        if not candidate:
            return None
        try:
            numeric = float(candidate)
        except ValueError:
            return None
        return numeric if math.isfinite(numeric) else None
    return None

In [ ]:
# Finalize the profiling job by persisting payload artifacts and metadata updates.
import os
import re


if "_ensure_notebook_stage" not in globals():
    raise RuntimeError("Profiling notebook Cells 1-6 must run sequentially; run Cells 2-5 before finalizing.")


_ensure_notebook_stage("finalization")


print(f"Finalizing profiling run {profile_run_id}...")




def _guess_profiling_payload():
    for key in ("profiling_payload", "profile_payload", "results_payload", "payload", "raw_payload"):
        candidate = globals().get(key)
        if isinstance(candidate, (dict, list)):
            return candidate
    return None




def _sanitize_segment(value, fallback):
    cleaned = (value or fallback or "").strip() or fallback
    return re.sub(r"[^a-zA-Z0-9_.-]", "_", cleaned)




def _default_artifact_path():
    base_root = (payload_base_path or DEFAULT_PRIVATE_PAYLOAD_ROOT or "dbfs:/tmp/conversioncentral/profiles").rstrip("/")
    group_segment = _sanitize_segment(table_group_id, "table_group")
    run_segment = _sanitize_segment(profile_run_id, "profile_run")
    return f"{base_root}/{group_segment}/{run_segment}.json"




def _materialize_payload_artifact(payload_obj, target_path):
    destination = target_path
    if destination.startswith("/dbfs/"):
        destination = f"dbfs:/{destination[6:]}"
    if destination.startswith("dbfs:/"):
        redirected = _redirect_dbfs_path(destination)
        if not redirected:
            return None
        destination = redirected
    encoded = _encode_payload_json(payload_obj) or json.dumps(payload_obj, default=str)
    if destination.startswith("dbfs:/") or destination.startswith("file:/"):
        _mkdirs_if_supported(destination)
        dbutils.fs.put(destination, encoded, True)
        return destination
    directory = os.path.dirname(destination)
    if directory:
        os.makedirs(directory, exist_ok=True)
    with open(destination, "w", encoding="utf-8") as handle:
        handle.write(encoded)
    return destination




results_payload = _guess_profiling_payload()
payload_location = payload_path or None
storage_mode = _resolve_payload_storage_mode()
artifact_mode_requested = _payload_storage_is_artifact(storage_mode)


if results_payload is not None and artifact_mode_requested:
    target_path = payload_location or _default_artifact_path()
    resolved_location = _materialize_payload_artifact(results_payload, target_path)
    if resolved_location:
        payload_location = resolved_location
        print(f"Persisted profiling payload artifact to {payload_location}.")
    else:
        print(f"Unable to persist payload artifact to {target_path}; continuing with inline payload only.")
elif artifact_mode_requested and not payload_location:
    print("Payload storage mode requires an artifact path; defaulting once payload data is available.")


if payload_location and payload_location.startswith("/dbfs/"):
    payload_location = f"dbfs:/{payload_location[6:]}"


if results_payload is None and not payload_location:
    raise RuntimeError(
        "Profiling payload missing. Re-run the profiling cells for run "
        f"{profile_run_id} or supply the payload_path widget so detail tables can be persisted.",
    )


_persist_results_to_metadata(results_payload, payload_location)
print("Profiling metadata persistence completed.")